### Coursework coding instructions (please also see full coursework spec)

Please choose if you want to do either Task 1 or Task 2. You should write your report about one task only.

For the task you choose you will need to do two approaches:
  - Approach 1, which can use use pre-trained embeddings / models
  - Approach 2, which should not use any pre-trained embeddings or models
We should be able to run both approaches from the same colab file

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks

#### Reproducibility:
  - Your .README file should explain how to replicate the different experiments mentioned in your report

Good luck! We are really looking forward to seeing your reports and your model code!

In [ ]:
# Imports
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import torch.optim as optim
import codecs
import tqdm

In [ ]:
# You will need to download any word embeddings required for your code, e.g.:

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

# For any packages that Colab does not provide auotmatically you will also need to install these below, e.g.:

#! pip install torch

--2021-02-11 02:06:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-11 02:06:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-11 02:06:42--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!wget https://www.cs.rochester.edu/u/nhossain/humicroedit/semeval-2020-task-7-data.zip
!unzip semeval-2020-task-7-data.zip
!rm semeval-2020-task-7-data.zip

--2021-02-11 02:17:55--  https://www.cs.rochester.edu/u/nhossain/humicroedit/semeval-2020-task-7-data.zip
Resolving www.cs.rochester.edu (www.cs.rochester.edu)... 192.5.53.208
Connecting to www.cs.rochester.edu (www.cs.rochester.edu)|192.5.53.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 919538 (898K) [application/zip]
Saving to: ‘semeval-2020-task-7-data.zip’

semeval-2020-task-7 100%[===================>] 897.99K  2.13MB/s    in 0.4s    

2021-02-11 02:17:56 (2.13 MB/s) - ‘semeval-2020-task-7-data.zip’ saved [919538/919538]

Archive:  semeval-2020-task-7-data.zip
   creating: data/
   creating: data/task-1/
  inflating: data/task-1/.DS_Store   
  inflating: data/task-1/dev.csv     
  inflating: data/task-1/train.csv   
  inflating: data/.DS_Store          
   creating: data/task-2/
  inflating: data/task-2/.DS_Store   
  inflating: data/task-2/dev.csv     
  inflating: data/task-2/train.csv   


In [ ]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
# Load data
train_df = pd.read_csv('data/task-2/train.csv')
test_df = pd.read_csv('data/task-2/dev.csv')

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Number of epochs
epochs = 10

# Proportion of training data for train compared to dev
train_proportion = 0.8


#### Approach 1: Using pre-trained representations

In [ ]:
# We define our training loop
def train(train_iter, dev_iter, model, number_epoch):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """

    print("Training model.")

    for epoch in range(1, number_epoch+1):
        
        model.train()
        
        epoch_loss = 0
        epoch_correct = 0
        no_observations = 0  # Observations used for training so far

        for batch in train_iter:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            optimizer.zero_grad()
            loss = loss_fn(predictions, target)

            correct, __ = model_performance(np.argmax(predictions.detach().cpu().numpy(), axis=1), target.detach().cpu().numpy())

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct

        valid_loss, valid_acc, __, __ = eval(dev_iter, model)

        epoch_loss, epoch_acc = epoch_loss / no_observations, epoch_correct / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train Accuracy: {epoch_acc:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. Accuracy: {valid_acc:.2f} |')

In [ ]:
# We evaluate performance on our dev set
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_correct = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            feature, target = batch

            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            correct, __ = model_performance(np.argmax(pred, axis=1), trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_correct/no_observations, np.array(pred_all), np.array(trg_all)

In [ ]:
# How we print the model performance
def model_performance(output, target, print_output=False):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    correct_answers = (output == target)
    correct = sum(correct_answers)
    acc = np.true_divide(correct,len(output))

    if print_output:
        print(f'| Acc: {acc:.2f} ')

    return correct, acc

In [ ]:
# To create our vocab
def create_vocab(data):
    """
    Creating a corpus of all the tokens used
    """
    """
    tokenized_corpus = [] # Let us put the tokenized corpus in a list

    for sentence in data:

        tokenized_sentence = []

        for token in sentence.split(' '): # simplest split is

            tokenized_sentence.append(token)

        tokenized_corpus.append(tokenized_sentence)
    """
    # =============== my code =====================
    tokenized_corpus = get_tokenized_corpus(data)
    # =============== my code =====================

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence in tokenized_corpus:

        for token in sentence:

            if token not in vocabulary:

                if True:
                    vocabulary.append(token)

    return vocabulary, tokenized_corpus

In [ ]:
# Used for collating our observations into minibatches:
def collate_fn_padd(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''

    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [len(f) for f, l in batch]

    seq_tensor = torch.zeros((len(batch), max(batch_features_len))).long()

    for idx, (seq, seqlen) in enumerate(zip(batch_features, batch_features_len)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

    batch_labels = torch.LongTensor(batch_labels)

    return seq_tensor, batch_labels

# We create a Dataset so we can create minibatches
class Task2Dataset(Dataset):

    def __init__(self, train_data, labels):
        self.x_train = train_data
        self.y_train = labels

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, item):
        return self.x_train[item], self.y_train[item]

In [ ]:

class BiLSTM_classification(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, batch_size, device):
        super(BiLSTM_classification, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.device = device
        self.batch_size = batch_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2label = nn.Linear(hidden_dim * 2, 3)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers * num_directions, minibatch_size, hidden_dim)
        return torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device), \
               torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device)

    def forward(self, sentence):
        embedded = self.embedding(sentence)
        embedded = embedded.permute(1, 0, 2)

        lstm_out, self.hidden = self.lstm(
            embedded.view(len(embedded), self.batch_size, self.embedding_dim), self.hidden)

        out = self.hidden2label(lstm_out[-1])
        return out

In [ ]:
import re
def replace_token(df):
  df["new1"] = df.apply(
      lambda x: re.sub(r"<.+/>", x["edit1"], x["original1"]), axis=1)
  df["new2"] = df.apply(
      lambda x: re.sub(r"<.+/>", x["edit2"], x["original2"]), axis=1)
def remove_token(train_df):
  train_df["new1"] = train_df.apply(
      lambda x: re.sub(r'[0-9]+', '', x["new1"]), axis=1)
  train_df["new2"] = train_df.apply(
      lambda x: re.sub(r'[0-9]+', '', x["new2"]), axis=1)

In [ ]:
def get_tokenized_corpus(corpus):
  tokenized_corpus_PR = []  # PR = punctuation removed
  re_punctuation_string = '[\s,/.\'\"\[\]|#]'

  for sentence in corpus:
    tokenized_sentence_PR = re.split(re_punctuation_string, sentence)
    tokenized_sentence_PR = list(filter(None, tokenized_sentence_PR)) # remove empty
    tokenized_sentence_PR = list(
        filter(lambda x: [word for word in x if word not in stop_words],
               tokenized_sentence_PR))
    tokenized_corpus_PR.append(tokenized_sentence_PR)
  
  return tokenized_corpus_PR

In [ ]:
train_df['new1'].loc[50:70]

50    ' Get ready Russia ' : Trump issues warning on...
51    ' I really believe ' Putin ' means it ' when h...
52    ' I really believe ' Putin ' means it ' when h...
53    ' I really doubt ' Putin ' means it ' when he ...
54    ' Ironclad ' : In Nearly Hour-Long Phone Call ...
55    ' Ironclad ' : In Nearly Hour-Long Phone Call ...
56    ' Ironclad ' : In Nearly Hour-Long Phone Call ...
57    ' Is he confused or are you confused ? ' : Sea...
58    ' Is he confused or are you confused ? ' : Sea...
59    ' Is he confused or are you confused ? ' : Sea...
60    ' It 's all explosive ' : Michael Wolff on dyn...
61    ' It 's called VOICE ' : Trump announces immig...
62    ' It 's called VOICE ' : Trump announces immig...
63    ' It 's called stupid ' : Trump announces immi...
64    ' I think he 'll be just fine ' : Trump hints ...
65    ' I think he 'll be just fine ' : Trump hints ...
66    ' I think he 'll be just fine ' : Trump hints ...
67    ' I was very angry ' at Trump , says Myesh

In [ ]:
def remove_invalid_headline(vectorized_seqs):
  valid_seq = []
  valid_idx = []
  for idx, seq in enumerate(vectorized_seqs):
    if len(seq) > 3:
      valid_seq.append(seq)
      valid_idx.append(idx)
  return valid_seq, valid_idx

In [ ]:
## Approach 1 code, using functions defined above:

# We set our training data and test data
#training_data = train_df['original1']
#test_data = test_df['original1']

#====================== my code ============================
replace_token(train_df)
remove_token(train_df)
training_data = train_df['new1']
replace_token(test_df)
test_data = test_df['new1']
#====================== my code ============================

# Creating word vectors
training_vocab, training_tokenized_corpus = create_vocab(training_data)
test_vocab, test_tokenized_corpus = create_vocab(test_data)

# Creating joint vocab from test and train:
joint_vocab, joint_tokenized_corpus = create_vocab(pd.concat([training_data, test_data]))

print("Vocab created.")

# We create representations for our tokens
wvecs = [] # word vectors
word2idx = [] # word2index
idx2word = []

# This is a large file, it will take a while to load in the memory!
with codecs.open('glove.6B.100d.txt', 'r','utf-8') as f:
  index = 1
  for line in f.readlines():
    # Ignore the first line - first line typically contains vocab, dimensionality
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in joint_vocab:
          (word, vec) = (word,
                     list(map(float,line.strip().split()[1:])))
          wvecs.append(vec)
          word2idx.append((word, index))
          idx2word.append((index, word))
          index += 1


wvecs = np.array(wvecs)
word2idx = dict(word2idx)
idx2word = dict(idx2word)

vectorized_seqs = [[word2idx[tok] for tok in seq if tok in word2idx] for seq in training_tokenized_corpus]
#====================== my code ============================
vectorized_seqs, valid_idx = remove_invalid_headline(vectorized_seqs)
#====================== my code ============================

INPUT_DIM = len(word2idx)
EMBEDDING_DIM = 100
BATCH_SIZE = 32

model = BiLSTM_classification(EMBEDDING_DIM, 50, INPUT_DIM, BATCH_SIZE, device)
print("Model initialised.")

model.to(device)
# We provide the model with our embeddings
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))

feature = vectorized_seqs

# 'feature' is a list of lists, each containing embedding IDs for word tokens
#train_and_dev = Task2Dataset(feature, train_df['label'])
train_and_dev = Task2Dataset(feature, train_df['label'][valid_idx].reset_index(drop=True))

train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples

train_dataset, dev_dataset = random_split(train_and_dev,
                                           (train_examples,
                                            dev_examples))


train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)

print("Dataloaders created.")

loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs)

Vocab created.
Model initialised.
Dataloaders created.
Training model.
| Epoch: 01 | Train Loss: 0.97 | Train Accuracy: 0.44 |         Val. Loss: 0.97 | Val. Accuracy: 0.45 |
| Epoch: 02 | Train Loss: 0.96 | Train Accuracy: 0.46 |         Val. Loss: 0.97 | Val. Accuracy: 0.45 |
| Epoch: 03 | Train Loss: 0.95 | Train Accuracy: 0.46 |         Val. Loss: 0.97 | Val. Accuracy: 0.48 |
| Epoch: 04 | Train Loss: 0.88 | Train Accuracy: 0.61 |         Val. Loss: 1.00 | Val. Accuracy: 0.50 |
| Epoch: 05 | Train Loss: 0.76 | Train Accuracy: 0.70 |         Val. Loss: 1.07 | Val. Accuracy: 0.52 |
| Epoch: 06 | Train Loss: 0.68 | Train Accuracy: 0.74 |         Val. Loss: 1.19 | Val. Accuracy: 0.52 |
| Epoch: 07 | Train Loss: 0.63 | Train Accuracy: 0.76 |         Val. Loss: 1.21 | Val. Accuracy: 0.51 |
| Epoch: 08 | Train Loss: 0.58 | Train Accuracy: 0.78 |         Val. Loss: 1.36 | Val. Accuracy: 0.51 |
| Epoch: 09 | Train Loss: 0.53 | Train Accuracy: 0.78 |         Val. Loss: 1.35 | Val. Accuracy: 

#### Approach 2: No pre-trained representations

In [ ]:
train_and_dev = train_df['edit1']

training_data, dev_data, training_y, dev_y = train_test_split(train_df['edit1'], train_df['label'],
                                                                        test_size=(1-train_proportion),
                                                                        random_state=42)

# We train a Tf-idf model
count_vect = CountVectorizer(stop_words='english')
train_counts = count_vect.fit_transform(training_data)
transformer = TfidfTransformer().fit(train_counts)
train_counts = transformer.transform(train_counts)
naive_model = MultinomialNB().fit(train_counts, training_y)

# Train predictions
predicted_train = naive_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = naive_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")

sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)


Train performance:
| Acc: 0.73 

Dev performance:
| Acc: 0.52 


#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + 1  # 1 is most common class
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, torch.tensor(dev_y.values), True)


Baseline performance:
| Acc: 0.45 
